<a href="https://colab.research.google.com/github/pscabral/IFMASOFTEX_1/blob/main/02C%C3%B3pia_de_Yolo8_v1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mega.py

from mega import Mega

# Crie uma instância da classe Mega
mega = Mega()

# Cole o link completo do Mega.nz (incluindo a chave)
url = "https://mega.nz/file/tndjHT4I#H0JmcKeZ-Kzs8xEsyf6FwrIsCXNydTXEEeCx3_ZAK98"

# Use o link completo para fazer o download
mega.download_url(url)

!unzip Eggs.v6i.yolov8.zip

In [ ]:
!pip install ultralytics

In [ ]:
!pip install albumentations

In [4]:
import os
import random
import shutil
from pathlib import Path

import pandas as pd
from PIL import Image
import cv2
from ultralytics import YOLO
from IPython.display import Video
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import trange, tqdm
import warnings
import glob
import albumentations as A

from shutil import copyfile
from matplotlib import animation, rc
rc('animation', html='jshtml')


In [6]:
img_folder_path = './content/Eggs.v6i/train/images/'
annot_folder_path = './content/Eggs.v6i/train/labels/'

# main function to augment images
def transformer(img_folder_path, annot_folder_path):

    img_paths = Path(img_folder_path).glob('*.jpg')

    for img_path in img_paths:

        img_name = str(img_path).split('/')[-1].split('.')[:-1]
        img_name = '.'.join(img_name)
        label_name = img_name + '.txt'

        label_path = os.path.join(annot_folder_path, label_name)

        result = horizontalFlip(str(img_path), label_path, img_name)

    return


def horizontalFlip(img_path, label_path, save_name):

    # setup transformer
    transformHorizontalFlip = A.Compose([A.HorizontalFlip(always_apply=True)],
                      bbox_params=A.BboxParams(format='yolo'))

    # annotation file to list
    with open(label_path) as f:
        lines = f.readlines()

    bbox = []
    for line in lines:
        l = line.split(' ')
        bbox.append([float(l[1]), float(l[2]), float(l[3]), float(l[4]), str(l[0])])

    img = cv2.imread(img_path)

    try:
        trns = transformHorizontalFlip(image = img, bboxes=bbox)

        # transformation results
        img_transformed = trns['image']
        boxes = trns['bboxes']

        # save image and annotation file in YOLO format
        cv2.imwrite('./content/Eggs.v6i/train/images/'+ save_name + '_hf.jpg', img_transformed)

        with open('/content/Eggs.v6i/train/labels/'+ save_name + '_hf.txt' , 'w') as f:
            for box in boxes:
                f.write(str(box[-1]))
                f.write(' '+str(box[0]))
                f.write(' '+str(box[1]))
                f.write(' '+str(box[2]))
                f.write(' '+str(box[3]))
                f.write('\n')

        return

    except:
        print('Failed:', save_name)
        return


transformer(img_folder_path, annot_folder_path)

In [ ]:
Image_dir = '/content/Eggs.v6i/train/images'

num_samples = 9
image_files = os.listdir(Image_dir)

# Randomly select num_samples images
rand_images = random.sample(image_files, num_samples)

fig, axes = plt.subplots(3, 3, figsize=(11, 11))

for i in range(num_samples):
    image = rand_images[i]
    ax = axes[i // 3, i % 3]
    ax.imshow(plt.imread(os.path.join(Image_dir, image)))
    ax.set_title(f'Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [8]:
image = cv2.imread("/content/Eggs.v6i/valid/images/0056_jpg.rf.459245cb18fa0daa6394f367e5e70940.jpg")
h, w, c = image.shape
print(f"The image has dimensions {w}x{h} and {c} channels.")

The image has dimensions 640x640 and 3 channels.


In [ ]:
model = YOLO("yolov8n.pt")

# Use the model to detect object
image = "/content/Eggs.v6i/valid/images/0056_jpg.rf.459245cb18fa0daa6394f367e5e70940.jpg"
result_predict = model.predict(source = image, imgsz=(640))

# show results
plot = result_predict[0].plot()
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
display(Image.fromarray(plot))

In [ ]:
Final_model = YOLO('yolov8n.pt')
Result_Final_model = Final_model.train(data="/content/Eggs.v6i/data.yaml", name = 'data', epochs=20, patience=20, batch = 16, cache = True, imgsz=640, iou = 0.5, augment=True, degrees=25.0, fliplr=0.0, lr0=0.0001, optimizer='Adam')


In [46]:
print('result of training')
tpaths2=[]
for dirname, _, filenames in os.walk('/content/runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)
print(tpaths2[0])

result of training
/content/runs/detect/train/F1_curve.png


In [ ]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(25,10))
    plt.imshow(image)
    plt.show()

In [13]:
print('Predict')
best_path0='/content/runs/detect/train/weights/best.pt'
source0='/content/syntheticfruit.v1i/valid/images'

Predict


In [14]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths[0])

/content/syntheticfruit.v1i/valid/images/0_jpg.rf.0e69ec43c53fe6e84409ab68377b3e1d.jpg


In [15]:
model2 = YOLO(best_path0)
print('yolo task=detect mode=predict')

yolo task=detect mode=predict


In [ ]:
!yolo task=detect mode=predict model={best_path0} conf=0.4 source={source0}

In [ ]:
print('Result of Prediction')
results = model2.predict(source0,conf=0.4)
print(len(results))

In [ ]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri=pd.DataFrame(results[i].boxes.data).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i']
display(PBOX)

In [39]:
class_map = {
    0: 'Apple',
    1: 'Apricot',
    2: 'Avocado',
    3: 'Banana',
    4: 'Beetroot',
    5: 'Blueberry',
    6: 'Cactus',
    7: 'Cantaloupe',
    8: 'Carambula',
    9: 'Cauliflower',
    10: 'Cherry',
    11: 'Chestnut',
    12: 'Clementine',
    13: 'Cocos',
    14: 'Dates',
    15: 'Eggplant',
    16: 'Ginger',
    17: 'Granadilla',
    18: 'Grape',
    19: 'Grapefruit',
    20: 'Guava',
    21: 'Hazelnut',
    22: 'Huckleberry',
    23: 'Kaki',
    24: 'Kiwi',
    25: 'Kohlrabi',
    26: 'Kumquats',
    27: 'Lemon',
    28: 'Limes',
    29: 'Lychee',
    30: 'Mandarine',
    31: 'Mango',
    32: 'Mangostan',
    33: 'Maracuja',
    34: 'Melon',
    35: 'Mulberry',
    36: 'Nectarine',
    37: 'Nut',
    38: 'Onion',
    39: 'Orange',
    40: 'Papaya',
    41: 'Passion',
    42: 'Peach',
    43: 'Pear',
    44: 'Pepino',
    45: 'Pepper',
    46: 'Physalis',
    47: 'Pineapple',
    48: 'Pitahaya',
    49: 'Plum',
    50: 'Pomegranate',
    51: 'Pomelo',
    52: 'Potato',
    53: 'Quince',
    54: 'Rambutan',
    55: 'Raspberry',
    56: 'Redcurrant',
    57: 'Salak',
    58: 'Strawberry',
    59: 'Tamarillo',
    60: 'Tangelo',
    61: 'Tomato',
    62: 'Walnut'
}

print(class_map[0])


Apple


In [ ]:
PBOX['class'] = PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX = PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

In [41]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.loc[i,'class']
            x=int(box.loc[i,'x'])
            y=int(box.loc[i,'y'])
            x2=int(box.loc[i,'x2'])
            y2=int(box.loc[i,'y2'])
            #print(label,x,y,x2,y2)
            cv2.putText(image, f'{label}', (x, int(y-4)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0),2)
            cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),2) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [42]:
def create_animation(ims):
    fig=plt.figure(figsize=(12,8))
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    text = plt.text(0.05, 0.05, f'Slide {0}', transform=fig.transFigure, fontsize=14, color='blue')
    plt.axis('off')
    plt.close()

    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        text.set_text(f'Slide {20+i}')
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000)

In [43]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

  0%|          | 0/1200 [00:00<?, ?it/s]

In [ ]:
create_animation(images2)

In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO('/content/runs/detect/train/weights/best.pt')

def run():

    model.predict(
        source="./pepino01.jpg",
        conf=0.25,
        save=True,
        show=True,
        save_crop=True
    )

if __name__ == '__main__':
    run()